In [5]:
!pip install python-docx fitz openpyxl spacy
!python -m spacy download en_core_web_sm
!pip install PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 93.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 86.7 MB/s eta 0:00:00


In [8]:
import os
import fitz  # PyMuPDF
import docx
import pandas as pd
import spacy
from google.colab import files

nlp = spacy.load("en_core_web_sm")

resumes = []
results = []
resume_texts = []

def upload_resumes():
    uploaded = files.upload()
    for filename in uploaded.keys():
        resumes.append(filename)
    print(f"Uploaded {len(resumes)} resume(s).")

def extract_text_from_resume(file_path):
    text = ""
    if file_path.endswith(".pdf"):
        with fitz.open(file_path) as doc:
            for page in doc:
                text += page.get_text()
    elif file_path.endswith(".docx"):
        doc = docx.Document(file_path)
        for para in doc.paragraphs:
            text += para.text + "\n"
    return text.strip()

def extract_text():
    if not resumes:
        print("Please upload resumes first!")
        return

    global resume_texts
    resume_texts = []
    for file in resumes:
        text = extract_text_from_resume(file)
        resume_texts.append((file, text))
    print("Text extracted from all resumes.")

def match_with_job():
    if not resume_texts:
        print("Please extract text from resumes first!")
        return

    job_description = input("Enter job description or required skills (comma-separated): ")
    job_keywords = [kw.strip().lower() for kw in job_description.split(",")]

    global results
    results = []
    for file, text in resume_texts:
        doc = nlp(text)
        tokens = [token.text.lower() for token in doc if not token.is_stop and token.is_alpha]
        matched = [kw for kw in job_keywords if kw in tokens]
        match_percent = round((len(matched) / len(job_keywords)) * 100, 2)
        results.append({
            "File": file,
            "Matched Skills": ", ".join(matched),
            "Match %": match_percent
        })
    print("Matching complete.")

def show_results():
    if not results:
        print("No results to show. Please run matching first.")
        return

    df = pd.DataFrame(results)
    print("\nMatch Results:")
    print(df.to_string(index=False))

def export_to_excel():
    if not results:
        print("No results to export.")
        return

    df = pd.DataFrame(results)
    file_name = "resume_match_results.xlsx"
    df.to_excel(file_name, index=False)
    print(f"Results exported to '{file_name}'")
    files.download(file_name)

def menu():
    while True:
        print("\nResume Scanner - Main Menu")
        print("1. Upload Resume(s)")
        print("2. Extract Text from Resume(s)")
        print("3. Match with Job Description")
        print("4. Show Results and Recommendations")
        print("5. Export Results to Excel")
        print("6. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            upload_resumes()
        elif choice == '2':
            extract_text()
        elif choice == '3':
            match_with_job()
        elif choice == '4':
            show_results()
        elif choice == '5':
            export_to_excel()
        elif choice == '6':
            print("Exiting program.")
            break
        else:
            print("Invalid choice. Please try again.")

# Run the menu
menu()



Resume Scanner - Main Menu
1. Upload Resume(s)
2. Extract Text from Resume(s)
3. Match with Job Description
4. Show Results and Recommendations
5. Export Results to Excel
6. Exit
Enter your choice: 6
Exiting program.
